In [1]:
# Importing stock ml libraries
import numpy as np
import pandas as pd
from sklearn import metrics
from tqdm import tqdm
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig,AutoTokenizer,AutoModel,ElectraModel



/media/cfs/wangran108/.pylib/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
!pip list | grep elast

elastic-transport       8.4.0
elasticsearch           6.3.1


In [2]:
# 72 75 77 78 84 85  89 93 98 104 106 107 108 109 111 114  115   118 121 125 131 138 139 149 150 155 164 165 170 173 177 182 188 200 205 206 220 228

In [2]:
# # Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
# device='cpu'

In [3]:
df = pd.read_excel("./各类型问题标注350条的_价格去掉纯数值询问.xlsx")
df['label']=df['label'].fillna('').astype('str')


tmp1=df[df.label!='0'][~df.label.str.contains(',')]

sent1=tmp1.sent.tolist()
label1=tmp1.label.tolist()
tmp2=tmp1.sample(len(tmp1))
sent2=tmp2.sent.tolist()
label2=tmp2.label.tolist()
sent3=[sent1[i]+sent2[i] for i in range(len(sent1))]
label3=[label1[i]+','+label2[i] for i in range(len(label1))]
df_added=pd.DataFrame()
df_added['sent']=sent3
df_added['label']=label3

df=pd.concat([df,df_added])

df['label2']=[np.zeros(12) for i in range(df.shape[0])]


def func(x):
    labels=x['label'].split(',')
    labels=[int(labels[i]) for i in range(len(labels))]
    for each in labels:
        x['label2'][each]=1
    return x
df=df.apply(lambda row :func(row),axis=1)
df.columns=['comment_text','2','list']
df0=df[df['2']==0]
# df=df.drop(df0.index[:40])
# df.columns=['comment_text', 'list']
new_df = df[['comment_text', 'list']].copy()

print(new_df.shape)
new_df.tail()

/home/wangran108/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


(1345, 2)


,comment_text,list
611,您这边有发货需要吗，女士是的是吧，那我就哦，那我就给您标记广州市天河区让这附近的销售经理给您...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
612,来电是关于我们京东物流有优惠政策，最大可以给您申请六折优惠，您看有这方面的需要吗？就是您所有...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
613,后期咱们这边需不需要冷链服务呀，就是看您这边嗯，那您发货的就是物品一单，就是比如说配件儿的价...,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, ..."
614,啊，就是咱这边就是您有发货的需要吗？就稍后给您报个价，因为咱这个京东有这个保价服务嘛布料吗哦...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
615,嗯，那咱们这个货品重量的话都是在一公斤以内的不是很重的这种是吧？呃，您平时都发哪些快递呢？,"[0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."


In [4]:
# Sections of config

# Defining some key variables that will be used later on in the training
MAX_LEN = 120
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 1
LEARNING_RATE = 2e-05
tokenizer = BertTokenizer.from_pretrained('/home/wangran108/code/model_file/chinese-electra-180g-large-discriminator')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'ElectraTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [5]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.comment_text = dataframe.comment_text
        self.targets = self.data.list
        self.max_len = max_len

    def __len__(self):
        return len(self.comment_text)

    def __getitem__(self, index):
        comment_text = str(self.comment_text[index])
        comment_text = " ".join(comment_text.split())

        inputs = self.tokenizer.encode_plus(
            comment_text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }
        

In [6]:
# Creating the dataset and dataloader for the neural network

train_size = 1.0
train_dataset=new_df.sample(frac=train_size,random_state=20230208)
test_dataset=new_df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)


print("FULL Dataset: {}".format(new_df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN)
testing_set = CustomDataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (1345, 2)
TRAIN Dataset: (1345, 2)
TEST Dataset: (0, 2)


In [7]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }


training_loader = DataLoader(training_set, **train_params)


In [9]:
# test_params = {'batch_size': VALID_BATCH_SIZE,
#                 'shuffle': True,
#                 'num_workers': 0
#                 }

# testing_loader = DataLoader(testing_set, **test_params)

In [8]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = transformers.ElectraModel.from_pretrained('/home/wangran108/code/model_file/chinese-electra-180g-large-discriminator')
        self.l2 = torch.nn.Dropout(0.5)
        self.l3 = torch.nn.Linear(1024, 12)
        self.dense = torch.nn.Linear(1024, 1024)
        self.activation =torch.nn.Tanh()
        # self.pooler=torch.mean()
    def forward(self, ids, mask, token_type_ids):
        # _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids,return_dict=False)
        hidden_states= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids,return_dict=True).last_hidden_state
        first_token_tensor = hidden_states[:, 0]
        pooled_output = self.dense(first_token_tensor)
        pooled_output = self.activation(pooled_output)
        # output_1=self.l1(pooled_output)
        # # output_1=torch.mean(output_1)
        output_2 = self.l2(pooled_output)
        output = self.l3(output_2)
        
        
        # output=self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids,return_dict=False)[0]
        return output

model = BERTClass()


Some weights of the model checkpoint at /home/wangran108/code/model_file/chinese-electra-180g-large-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
model.to('cuda')
model=torch.nn.DataParallel(model )

In [10]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [11]:
optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE)

In [12]:
def train(epoch):
    model.train()
    for _,data in enumerate(tqdm(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _%5000==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        torch.cuda.empty_cache()
        

In [13]:
# def validation_test(epoch):
#     model.eval()
#     fin_targets=[]
#     fin_outputs=[]
#     with torch.no_grad():
#         for _, data in enumerate(testing_loader, 0):
#             ids = data['ids'].to(device, dtype = torch.long)
#             mask = data['mask'].to(device, dtype = torch.long)
#             token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
#             targets = data['targets'].to(device, dtype = torch.float)
#             outputs = model(ids, mask, token_type_ids)
            
#             # print(outputs)
#             # print(np.argmax(outputs.cpu().detach().numpy().tolist(),axis=-1))
#             # print(np.argmax(targets.cpu().detach().numpy().tolist(),axis=-1))
#             fin_targets.extend(np.argmax(targets.cpu().detach().numpy().tolist(),axis=-1))
#             fin_outputs.extend(np.argmax(outputs.cpu().detach().numpy().tolist(),axis=-1))
#     return fin_outputs, fin_targets

def validation(epoch,loader=training_loader):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
            # fin_targets.extend(np.argmax(targets.cpu().detach().numpy().tolist(),axis=-1))
            # fin_outputs.extend(np.argmax(outputs.cpu().detach().numpy().tolist(),axis=-1))
    return fin_outputs, fin_targets

In [20]:
train(1)

  0%|          | 0/43 [00:00<?, ?it/s]/home/wangran108/anaconda3/envs/py36/lib/python3.6/site-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Epoch: 1, Loss:  0.07176007330417633


100%|██████████| 43/43 [01:15<00:00,  1.75s/it]


In [14]:
for epoch in range(4):
    train(epoch)

  0%|          | 0/43 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/wangran108/anaconda3/envs/py36/lib/python3.6/site-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Epoch: 0, Loss:  0.7239732146263123


  0%|          | 0/43 [00:00<?, ?it/s]

Epoch: 1, Loss:  0.3225160241127014


  0%|          | 0/43 [00:00<?, ?it/s]

Epoch: 2, Loss:  0.24871677160263062


  0%|          | 0/43 [00:00<?, ?it/s]

Epoch: 3, Loss:  0.1372227519750595


100%|██████████| 43/43 [01:13<00:00,  1.70s/it]


In [21]:
for epoch in range(EPOCHS):
    outputs, targets = validation(epoch,training_loader)
    outputs = np.array(outputs) >= 0.5
    outputs=outputs.astype('int')
    targets=np.array(targets).astype('int')
    
    
    accuracy = metrics.accuracy_score(targets, outputs)
    f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
    f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
    print(f"Accuracy Score = {accuracy}")
    print(f"F1 Score (Micro) = {f1_score_micro}")
    print(f"F1 Score (Macro) = {f1_score_macro}")
    torch.cuda.empty_cache()

Accuracy Score = 0.9858736059479554
F1 Score (Micro) = 0.9944282302998142
F1 Score (Macro) = 0.9941247820818416


In [22]:
torch.save(model.module, 'bert_qustion_multi7_6.pth')

In [5]:
model=torch.load('bert_qustion_multi5_2.pth')

In [31]:

model.eval()
inputs = tokenizer.encode_plus(
        '请问发快递',
        None,
        add_special_tokens=True,
        max_length=120,
        pad_to_max_length=True,
        return_token_type_ids=True
    )
ids = torch.tensor([inputs['input_ids']], dtype=torch.long).to(device, dtype = torch.long)
mask = torch.tensor([inputs['attention_mask']]).to(device, dtype = torch.long)
token_type_ids =torch.tensor([inputs['token_type_ids']]).to(device, dtype = torch.long)

outputs = model(ids, mask, token_type_ids)
print(torch.sigmoid(outputs))
# np.argmax(outputs.cpu().detach().numpy().tolist(),axis=-1)[0]
torch.topk(torch.sigmoid(outputs),3)[1][0].cpu().numpy()

tensor([[0.0265, 0.9401, 0.0306, 0.0242, 0.0287, 0.0323, 0.0392, 0.0201, 0.0293,
         0.0178, 0.0208, 0.0162]], device='cuda:0', grad_fn=<SigmoidBackward0>)


array([1, 6, 5])

In [ ]:
tokenizer.decode(inputs['input_ids'])

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(outputs,targets)

In [ ]:
torch.save(model.state_dict(), '/home/wangran108/code/Unilm/model_weights.pth')

In [ ]:
# 数据预处理  将对话里的标注转换成句子标注

In [ ]:
import sys
import os
ns=os.listdir('/home/wangran108/code/machine_quality_check/file/dialog_doc')


In [ ]:
ns

In [ ]:
import numpy as np
from transformers import T5Tokenizer,MT5ForConditionalGeneration

pretrain_path = '/home/wangran108/code/model_file/randengt5'
tokenizer=T5Tokenizer.from_pretrained(pretrain_path)


In [ ]:
model=MT5ForConditionalGeneration.from_pretrained(pretrain_path)


In [ ]:
# !pip install transformers==4.20
# !pip list
# !pip install torch==1.6

In [ ]:

sample={"context":"客服:嗯，行了解了，那您这边没有特殊的这个物流需求吧。顾客:特殊的物流需求是啥意思啊？客服:类似于整车呀，仓储呀。顾客:我一般用冷链",\
    "question":"顾客用到了特殊物流需求吗，是什么","idx":1}
plain_text='question:'+sample['question']+'knowledge:'+sample['context'][:120]

res_prefix=tokenizer.encode('answer'+'<extra_id_0></s>',add_special_token=False)
l_rp=len(res_prefix)

tokenized=tokenizer.encode(plain_text,truncation=True,max_length=200-2-l_rp)

tokenized+=res_prefix
import torch
# Generate answer
# 客服:嗯，行了解了，那您这边没有特殊的这个物流需求吧。顾客:特殊的物流需求是啥意思啊？客服:类似于整车呀，仓储呀。顾客:我一般用冷链
# 客服说“嗯，行了解了，那您这边没有特殊的这个物流需求吧”顾客说“特殊的物流需求是啥意思啊？”客服说“类似于整车呀，仓储呀。”顾客说“我一般用冷链”

In [ ]:
tokenizer.decode(tokenized)

In [ ]:



pred_ids = model.generate(input_ids=torch.LongTensor([tokenized]),max_length=10,do_sample=False,top_p=0.9)

In [ ]:
tokenizer.batch_decode(pred_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)


In [ ]:
execl_names=os.listdir("/home/wangran108/code/Unilm/excel/")

In [ ]:
########采样
sents=[]
labels=[]
def flag_map(x):
    x=x.strip()
    if x=='顾客:':
        return 0
    if x=='客服:':
        return 1
    return 2

for n in execl_names:
    
    df=pd.read_excel(f"/home/wangran108/code/Unilm/excel/{n}")
    tmp=df[df.label!=0]
    
    df['flag']=df['sent'].map(lambda x: flag_map(x))
    flag=df['flag'].tolist()
    for i in range(1,len(flag)):
        if flag[i-1]==1:
            flag[i]=1
    df['flag']=flag
    if tmp.shape[0]==0:
        continue
    df['len']=df['sent'].map(lambda x:len(x))
    tmp2=df[df.label==0][df.sent!='顾客:\n'][df.sent!='客服:\n'][df.flag==1][df.len>5].sample(3)
    # sents.extend(tmp.sent.tolist())
    sents.extend(tmp2.sent.tolist())

    # labels.extend(tmp.label.tolist())
    labels.extend(tmp2.label.tolist())
    
    

In [ ]:
tmpdf=pd.DataFrame()
tmpdf['sents']=sents
tmpdf['labels']=labels
tmpdf.to_excel('tmp.xlsx')

In [ ]:
df=pd.DataFrame()
df['sents']=sents
df['label']=labels